<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 2*
# Convolutional Neural Networks (CNNs)

# Assignment

- <a href="#p1">Part 1:</a> Pre-Trained Model
- <a href="#p2">Part 2:</a> Custom CNN Model
- <a href="#p3">Part 3:</a> CNN with Data Augmentation


You will apply three different CNN models to a binary image classification model using Keras. Classify images of Mountains (`./data/train/mountain/*`) and images of forests (`./data/train/forest/*`). Treat mountains as the positive class (1) and the forest images as the negative (zero). 

|Mountain (+)|Forest (-)|
|---|---|
|![](https://github.com/richardOlson/DS-Unit-4-Sprint-3-Deep-Learning/blob/master/module2-convolutional-neural-networks/data/train/mountain/art1131.jpg?raw=1)|![](https://github.com/richardOlson/DS-Unit-4-Sprint-3-Deep-Learning/blob/master/module2-convolutional-neural-networks/data/validation/forest/cdmc317.jpg?raw=1)|

The problem is relatively difficult given that the sample is tiny: there are about 350 observations per class. This sample size might be something that you can expect with prototyping an image classification problem/solution at work. Get accustomed to evaluating several different possible models.

# Pre - Trained Model
<a id="p1"></a>

Load a pretrained network from Keras, [ResNet50](https://tfhub.dev/google/imagenet/resnet_v1_50/classification/1) - a 50 layer deep network trained to recognize [1000 objects](https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt). Starting usage:

```python
import numpy as np

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model # This is the functional API

resnet = ResNet50(weights='imagenet', include_top=False)

```

The `include_top` parameter in `ResNet50` will remove the full connected layers from the ResNet model. The next step is to turn off the training of the ResNet layers. We want to use the learned parameters without updating them in future training passes. 

```python
for layer in resnet.layers:
    layer.trainable = False
```

Using the Keras functional API, we will need to additional additional full connected layers to our model. We we removed the top layers, we removed all preivous fully connected layers. In other words, we kept only the feature processing portions of our network. You can expert with additional layers beyond what's listed here. The `GlobalAveragePooling2D` layer functions as a really fancy flatten function by taking the average of each of the last convolutional layer outputs (which is two dimensional still). 

```python
x = resnet.output
x = GlobalAveragePooling2D()(x) # This layer is a really fancy flatten
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(resnet.input, predictions)
```

Your assignment is to apply the transfer learning above to classify images of Mountains (`./data/train/mountain/*`) and images of forests (`./data/train/forest/*`). Treat mountains as the positive class (1) and the forest images as the negative (zero). 

Steps to complete assignment: 
1. Load in Image Data into numpy arrays (`X`) 
2. Create a `y` for the labels
3. Train your model with pre-trained layers from resnet
4. Report your model's accuracy

## Load in Data

This surprisingly more difficult than it seems, because you are working with directories of images instead of a single file. This boiler plate will help you download a zipped version of the directory of images. The directory is organized into "train" and "validation" which you can use inside an `ImageGenerator` class to stream batches of images thru your model.  


### Download & Summarize the Data

This step is completed for you. Just run the cells and review the results. 

In [15]:
import tensorflow as tf
import os

_URL = 'https://github.com/LambdaSchool/DS-Unit-4-Sprint-3-Deep-Learning/blob/master/module2-convolutional-neural-networks/data.zip?raw=true'

path_to_zip = tf.keras.utils.get_file('./data.zip', origin=_URL, extract=True)
PATH = os.path.join(os.path.dirname(path_to_zip), 'data')

42172416/42170838 [==============================] - 0s 0us/step


In [0]:
train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

In [17]:
train_dir

'/root/.keras/datasets/./data/train'

In [0]:
train_mountain_dir = os.path.join(train_dir, 'mountain')  # directory with our training cat pictures
train_forest_dir = os.path.join(train_dir, 'forest')  # directory with our training dog pictures
validation_mountain_dir = os.path.join(validation_dir, 'mountain')  # directory with our validation cat pictures
validation_forest_dir = os.path.join(validation_dir, 'forest')  # directory with our validation dog pictures

In [0]:
num_mountain_tr = len(os.listdir(train_mountain_dir))
num_forest_tr = len(os.listdir(train_forest_dir))

num_mountain_val = len(os.listdir(validation_mountain_dir))
num_forest_val = len(os.listdir(validation_forest_dir))

total_train = num_mountain_tr + num_forest_tr
total_val = num_mountain_val + num_forest_val

In [20]:
print('total training mountain images:', num_mountain_tr)
print('total training forest images:', num_forest_tr)

print('total validation mountain images:', num_mountain_val)
print('total validation forest images:', num_forest_val)
print("--")
print("Total training images:", total_train)
print("Total validation images:", total_val)

total training mountain images: 254
total training forest images: 270
total validation mountain images: 125
total validation forest images: 62
--
Total training images: 524
Total validation images: 187


### Keras `ImageGenerator` to Process the Data

This step is completed for you, but please review the code. The `ImageGenerator` class reads in batches of data from a directory and pass them to the model one batch at a time. Just like large text files, this method is advantageous, because it stifles the need to load a bunch of images into memory. 

Check out the documentation for this class method: [Keras `ImageGenerator` Class](https://keras.io/preprocessing/image/#imagedatagenerator-class). You'll expand it's use in the third assignment objective.

In [0]:
# doing some of the imports
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing  import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.layers import  Dense, GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import  Layer, Dense
from tensorflow.keras.models import  Model
from tensorflow.keras import Sequential

In [0]:
batch_size = 16
epochs = 50
IMG_HEIGHT = 224
IMG_WIDTH = 224

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_image_generator = ImageDataGenerator(rescale=1./255, preprocessing_function=preprocess_input, shear_range=.2, zoom_range=.2, horizontal_flip=True) # Generator for our training data
validation_image_generator = ImageDataGenerator(rescale=1./255) # Generator for our validation data

In [47]:
train_data_gen = train_image_generator.flow_from_directory(batch_size=batch_size,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                           class_mode='binary')

Found 533 images belonging to 2 classes.


In [22]:
type(train_data_gen)

keras_preprocessing.image.directory_iterator.DirectoryIterator

In [48]:
val_data_gen = validation_image_generator.flow_from_directory(batch_size=batch_size,
                                                              directory=validation_dir,
                                                              target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                              class_mode='binary')

Found 195 images belonging to 2 classes.


## Instatiate Model

In [24]:
# instanciating the resnet
resnet = ResNet50(include_top=False, weights="imagenet", pooling="avg", input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))

94773248/94765736 [==============================] - 3s 0us/step


In [0]:
resnet.trainable = False

In [0]:
resnet.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [0]:
train_data_gen[0][1].shape

(16,)

In [0]:
type(train_data_gen[0][0])

numpy.ndarray

In [0]:
train_data_gen[0][1]

array([1., 0., 1., 0., 1., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0.],
      dtype=float32)

In [0]:
train_data_gen[0][0].shape

(16, 224, 224, 3)

In [0]:
# Will be adding a new layer to the the output of one
# for the classification
#adding 2 dense layers
firstDense = Dense(1024, activation="relu")
outPutLayer = Dense(1, activation="sigmoid")
onlyOutPut = Dense(1, activation="sigmoid")

In [0]:
# making the model that will only have the ouput layer added
onlyOutPutModel = Sequential([resnet, onlyOutPut])

In [0]:
# building the complete model
completeModel = Sequential([resnet, firstDense, outPutLayer])

In [0]:
onlyOutPutModel.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=["accuracy"], 
                        optimizer=tf.keras.optimizers.RMSprop(learning_rate=.0001))

In [0]:
# now doing the compiling of the model
completeModel.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=["accuracy"],
                      optimizer=tf.keras.optimizers.RMSprop(learning_rate=.0001))

In [0]:
completeModel.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
dense_5 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 1025      
Total params: 25,686,913
Trainable params: 2,099,201
Non-trainable params: 23,587,712
_________________________________________________________________


In [0]:
onlyOutPutModel.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 2049      
Total params: 23,589,761
Trainable params: 2,049
Non-trainable params: 23,587,712
_________________________________________________________________


In [0]:
x_train, y_train = next(train_data_gen)

In [0]:
x_train.shape

(16, 224, 224, 3)

In [0]:
# Getting the images out as a batch

In [0]:
# Doing the evaluation of with just changing the 
# output to what we wanted
# Doing the loop with the dataGenerator

the_losses, the_accuracy = onlyOutPutModel.evaluate(train_data_gen)


34/34 [==============================] - 77s 2s/step - loss: 0.7522 - accuracy: 0.4859


In [0]:
# Doing the evaluation again with the two layers for the 
# dense layers at the end
# Will just do one epoch
the_losses, the_accuracy = completeModel.evaluate(train_data_gen)

34/34 [==============================] - 76s 2s/step - loss: 0.7455 - accuracy: 0.4859


## Fit Model

In [0]:
# This is with just just running the model with only able to train the output 
# layer
history = onlyOutPutModel.fit(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size
)

Epoch 1/50
32/32 [==============================] - 8s 239ms/step - loss: 0.7737 - accuracy: 0.4870 - val_loss: 0.6468 - val_accuracy: 0.6534
Epoch 2/50
32/32 [==============================] - 7s 218ms/step - loss: 0.7539 - accuracy: 0.4810 - val_loss: 0.6535 - val_accuracy: 0.6250
Epoch 3/50
32/32 [==============================] - 7s 213ms/step - loss: 0.7368 - accuracy: 0.5090 - val_loss: 0.6513 - val_accuracy: 0.3466
Epoch 4/50
32/32 [==============================] - 7s 214ms/step - loss: 0.7131 - accuracy: 0.5070 - val_loss: 0.6568 - val_accuracy: 0.3466
Epoch 5/50
32/32 [==============================] - 7s 213ms/step - loss: 0.7035 - accuracy: 0.5010 - val_loss: 0.6613 - val_accuracy: 0.3409
Epoch 6/50
32/32 [==============================] - 7s 213ms/step - loss: 0.7023 - accuracy: 0.5170 - val_loss: 0.6647 - val_accuracy: 0.3295
Epoch 7/50
32/32 [==============================] - 7s 210ms/step - loss: 0.6970 - accuracy: 0.5050 - val_loss: 0.6716 - val_accuracy: 0.3466
Epoch 

In [0]:
# Will have one dense layer before the 
# output layer that can be trained
history = completeModel.fit(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size
)

Epoch 1/50
32/32 [==============================] - 8s 236ms/step - loss: 0.6965 - accuracy: 0.5190 - val_loss: 0.6810 - val_accuracy: 0.3409
Epoch 2/50
32/32 [==============================] - 7s 211ms/step - loss: 0.6932 - accuracy: 0.5190 - val_loss: 0.6859 - val_accuracy: 0.3409
Epoch 3/50
32/32 [==============================] - 7s 210ms/step - loss: 0.6913 - accuracy: 0.5130 - val_loss: 0.6592 - val_accuracy: 0.3182
Epoch 4/50
32/32 [==============================] - 7s 211ms/step - loss: 0.6900 - accuracy: 0.5210 - val_loss: 0.6799 - val_accuracy: 0.3636
Epoch 5/50
32/32 [==============================] - 7s 209ms/step - loss: 0.6887 - accuracy: 0.5190 - val_loss: 0.6415 - val_accuracy: 0.3580
Epoch 6/50
32/32 [==============================] - 7s 210ms/step - loss: 0.6872 - accuracy: 0.5170 - val_loss: 0.6546 - val_accuracy: 0.3523
Epoch 7/50
32/32 [==============================] - 7s 212ms/step - loss: 0.6816 - accuracy: 0.5090 - val_loss: 0.6212 - val_accuracy: 0.5909
Epoch 

In [0]:
# Will now try to do a fine tunning of the model
# length of the layers 
len(resnet.layers)

176

In [0]:
tune_after = 176 -10

In [0]:
# making all the layers trainable first
resnet.trainable = True

In [0]:
# going to fine tune the last 10 layers 
# Those that are before this point are set to not trainable
for layer in resnet.layers[:tune_after]:
  layer.trainable = False

In [0]:
resnet.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [0]:
# Making two layers that will be trainable also
myDense_1 = Dense(1024, activation="relu")
myDense_2 = Dense(1, activation="sigmoid")


In [0]:
# Will now rebuilt the model
fineTunedModel = Sequential([resnet,  onlyOutPut])

In [0]:
# now doing the compilation of the model
fineTunedModel.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=.0001), loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=["accuracy"])

In [0]:
# creating a callback to do early stopping
myCallback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3, )

In [0]:
# Training the model that has just the last 10 layers of the resnet 
# that can be trained.
fineTunedModel.fit(train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size, 
    callbacks = [myCallback]
)

Epoch 1/50
32/32 [==============================] - 8s 247ms/step - loss: 0.6648 - accuracy: 0.6228 - val_loss: 0.6192 - val_accuracy: 0.4943
Epoch 2/50
32/32 [==============================] - 7s 216ms/step - loss: 0.6176 - accuracy: 0.7924 - val_loss: 0.5652 - val_accuracy: 0.8295
Epoch 3/50
32/32 [==============================] - 7s 217ms/step - loss: 0.5917 - accuracy: 0.8403 - val_loss: 0.5343 - val_accuracy: 0.9261
Epoch 4/50
32/32 [==============================] - 7s 217ms/step - loss: 0.5671 - accuracy: 0.8862 - val_loss: 0.5454 - val_accuracy: 0.8295
Epoch 5/50
32/32 [==============================] - 7s 217ms/step - loss: 0.5756 - accuracy: 0.8703 - val_loss: 0.5248 - val_accuracy: 0.8636
Epoch 6/50
32/32 [==============================] - 7s 217ms/step - loss: 0.5611 - accuracy: 0.9022 - val_loss: 0.5730 - val_accuracy: 0.7614
Epoch 7/50
32/32 [==============================] - 7s 216ms/step - loss: 0.5639 - accuracy: 0.8782 - val_loss: 0.5020 - val_accuracy: 0.9318
Epoch 

In [0]:
# This is doing just the classification using the just the resnet model 
# with no other changes to the model
results = resnet.predict()

# Custom CNN Model

In this step, write and train your own convolutional neural network using Keras. You can use any architecture that suits you as long as it has at least one convolutional and one pooling layer at the beginning of the network - you can add more if you want. 

In [0]:
# Doing some imports
from tensorflow.keras.layers import Conv2D, Dense
from tensorflow.keras.layers import ZeroPadding2D , BatchNormalization, GlobalAveragePooling2D, MaxPool2D, InputLayer, ReLU

In [0]:
theModel = Sequential()

In [0]:
theModel.add(Conv2D(64, (3,3), padding="same", input_shape=(224,224,3)))
theModel.add(BatchNormalization())
theModel.add(ReLU())
theModel.add(ZeroPadding2D(padding=(2,2)))
theModel.add(MaxPool2D(pool_size=(2,2)))
theModel.add(Conv2D(32, (3,3), padding="same"))
theModel.add(BatchNormalization())
theModel.add(ReLU())
theModel.add(Conv2D(32, (3,3), padding="same"))
theModel.add(BatchNormalization())
theModel.add(ReLU())
theModel.add(GlobalAveragePooling2D())
theModel.add(Dense(1, activation="sigmoid"))


In [0]:
# Doing the compilation
theModel.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=.0001), loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=["accuracy"])

In [68]:
theModel.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_28 (Conv2D)           (None, 224, 224, 64)      1792      
_________________________________________________________________
batch_normalization_27 (Batc (None, 224, 224, 64)      256       
_________________________________________________________________
re_lu_27 (ReLU)              (None, 224, 224, 64)      0         
_________________________________________________________________
zero_padding2d_13 (ZeroPaddi (None, 228, 228, 64)      0         
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 114, 114, 64)      0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 114, 114, 32)      18464     
_________________________________________________________________
batch_normalization_28 (Batc (None, 114, 114, 32)     

In [69]:
history = theModel.fit(train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size, 
    #callbacks = [myCallback]
    )

Epoch 1/50
32/32 [==============================] - 7s 233ms/step - loss: 0.6424 - accuracy: 0.7086 - val_loss: 0.6499 - val_accuracy: 0.6080
Epoch 2/50
32/32 [==============================] - 7s 224ms/step - loss: 0.6165 - accuracy: 0.8363 - val_loss: 0.6423 - val_accuracy: 0.6648
Epoch 3/50
32/32 [==============================] - 7s 224ms/step - loss: 0.6032 - accuracy: 0.8603 - val_loss: 0.6590 - val_accuracy: 0.6420
Epoch 4/50
32/32 [==============================] - 7s 223ms/step - loss: 0.5945 - accuracy: 0.8723 - val_loss: 0.6570 - val_accuracy: 0.6477
Epoch 5/50
32/32 [==============================] - 7s 225ms/step - loss: 0.5927 - accuracy: 0.8962 - val_loss: 0.6642 - val_accuracy: 0.6420
Epoch 6/50
32/32 [==============================] - 7s 224ms/step - loss: 0.5881 - accuracy: 0.8962 - val_loss: 0.6602 - val_accuracy: 0.6477
Epoch 7/50
32/32 [==============================] - 7s 228ms/step - loss: 0.5911 - accuracy: 0.8922 - val_loss: 0.6557 - val_accuracy: 0.6534
Epoch 

In [0]:
theModel = Sequential()

In [0]:
# Will try the model again but add 2 Dense layers
theModel.add(Conv2D(64, (3,3), padding="same", input_shape=(224,224,3)))
theModel.add(BatchNormalization())
theModel.add(ReLU())
theModel.add(ZeroPadding2D(padding=(2,2)))
theModel.add(MaxPool2D(pool_size=(2,2)))
theModel.add(Conv2D(32, (3,3), padding="same"))
theModel.add(BatchNormalization())
theModel.add(ReLU())
theModel.add(Conv2D(32, (3,3), padding="same"))
theModel.add(BatchNormalization())
theModel.add(ReLU())
theModel.add(GlobalAveragePooling2D())
theModel.add(Dense(64, activation="relu"))
theModel.add(Dense(64, activation="relu"))
theModel.add(Dense(1, activation="sigmoid"))


In [0]:
# Compile Model
theModel.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=.0001), loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=["accuracy"])

In [73]:
# Fit Model
history = theModel.fit(train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size, 
    #callbacks = [myCallback]
    )

Epoch 1/50
32/32 [==============================] - 8s 237ms/step - loss: 0.6951 - accuracy: 0.7944 - val_loss: 0.6447 - val_accuracy: 0.3693
Epoch 2/50
32/32 [==============================] - 7s 231ms/step - loss: 0.6515 - accuracy: 0.8643 - val_loss: 0.6501 - val_accuracy: 0.6477
Epoch 3/50
32/32 [==============================] - 7s 231ms/step - loss: 0.6255 - accuracy: 0.8862 - val_loss: 0.6428 - val_accuracy: 0.6591
Epoch 4/50
32/32 [==============================] - 7s 229ms/step - loss: 0.6093 - accuracy: 0.8902 - val_loss: 0.6378 - val_accuracy: 0.6648
Epoch 5/50
32/32 [==============================] - 7s 230ms/step - loss: 0.5932 - accuracy: 0.9082 - val_loss: 0.6429 - val_accuracy: 0.6591
Epoch 6/50
32/32 [==============================] - 8s 239ms/step - loss: 0.5869 - accuracy: 0.9062 - val_loss: 0.6453 - val_accuracy: 0.6591
Epoch 7/50
32/32 [==============================] - 7s 230ms/step - loss: 0.5803 - accuracy: 0.9082 - val_loss: 0.6305 - val_accuracy: 0.6761
Epoch 

# Custom CNN Model with Image Manipulations

To simulate an increase in a sample of image, you can apply image manipulation techniques: cropping, rotation, stretching, etc. Luckily Keras has some handy functions for us to apply these techniques to our mountain and forest example. Simply, you should be able to modify our image generator for the problem. Check out these resources to help you get started: 

1. [Keras `ImageGenerator` Class](https://keras.io/preprocessing/image/#imagedatagenerator-class)
2. [Building a powerful image classifier with very little data](https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html)
 

# Resources and Stretch Goals

Stretch goals
- Enhance your code to use classes/functions and accept terms to search and classes to look for in recognizing the downloaded images (e.g. download images of parties, recognize all that contain balloons)
- Check out [other available pretrained networks](https://tfhub.dev), try some and compare
- Image recognition/classification is somewhat solved, but *relationships* between entities and describing an image is not - check out some of the extended resources (e.g. [Visual Genome](https://visualgenome.org/)) on the topic
- Transfer learning - using images you source yourself, [retrain a classifier](https://www.tensorflow.org/hub/tutorials/image_retraining) with a new category
- (Not CNN related) Use [piexif](https://pypi.org/project/piexif/) to check out the metadata of images passed in to your system - see if they're from a national park! (Note - many images lack GPS metadata, so this won't work in most cases, but still cool)

Resources
- [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385) - influential paper (introduced ResNet)
- [YOLO: Real-Time Object Detection](https://pjreddie.com/darknet/yolo/) - an influential convolution based object detection system, focused on inference speed (for applications to e.g. self driving vehicles)
- [R-CNN, Fast R-CNN, Faster R-CNN, YOLO](https://towardsdatascience.com/r-cnn-fast-r-cnn-faster-r-cnn-yolo-object-detection-algorithms-36d53571365e) - comparison of object detection systems
- [Common Objects in Context](http://cocodataset.org/) - a large-scale object detection, segmentation, and captioning dataset
- [Visual Genome](https://visualgenome.org/) - a dataset, a knowledge base, an ongoing effort to connect structured image concepts to language